In [1]:
import pandas as pd
import requests
import json

In [2]:
df = pd.read_excel('/home/andy/Downloads/Razor Model Testing Data.xlsx')

In [3]:
df.columns

Index(['#', 'Organization', 'Root Sentence', 'Intent words', 'Unnamed: 4',
       'Variation 1', 'Variation 2', 'Variation 3', 'Variation 4',
       'Variation 5', 'Variation 6', 'Variation 7', 'Variation 8',
       'Variation 9', 'Variation 10', 'Unnamed: 15', 'Unnamed: 16',
       'Expected Number of Detections'],
      dtype='object')

In [4]:
sentences=df['Root Sentence'].to_list()
cols = []
for i in range(10):
    cols.append("Variation {}".format(i+1))
for i,value in df[cols].iterrows():
    sentences.append(value[0])

In [5]:
final_sentences=[]
for i in sentences:
    if type(i) != float:
        final_sentences.append(i)

In [6]:
def send_request(sentence):
    url = "http://localhost:9999/detect_lq?sentence={}".format(sentence)
    response = requests.request("GET", url)
    return json.loads(response.text)

In [17]:
results = list(map(send_request,final_sentences))

In [8]:
cnt=0
for result in results:
    for item in result:
        if item['Is_lq']:
            cnt+=1

In [43]:
final_result=[]
for result in results:
    for item in result:
        try:
            if item["Is_lq"]:
                final_result.append({"Sentence":item['Sentence'],"Is_lq":"True","Q-type":item['Question_type']['tag'],"Context_noun":item['Context_noun'],"Context_verb":item['Context_Verb']})
            else:
                final_result.append({"Sentence":item['Sentence'],"Is_lq":"False"})
        except TypeError as exe:
            final_result.append({"Sentence":item['Sentence'],"Is_lq":"True","Q-type":"Not question","Context_noun":item['Context_noun'],"Context_verb":item['Context_Verb']})
            pass                  

In [44]:
final_df = pd.DataFrame(final_result)

In [45]:
final_df.to_excel("/home/andy/Desktop/Pattern_lq_test_result.xlsx")